## Extracting edge flow

In [1]:
# load data frame with transactions
import pandas as pd
import numpy as np
import pickle
import networkx as nx
from pprint import pprint
from datetime import datetime
from datetime import date 
from etl import data
from sklearn.metrics import mean_squared_error


df = data.df
df_ac = df.groupby('ac_location').count().sort_values('ct_id', ascending=False)
df_mc = df.groupby('mc_location').count().sort_values('ct_id', ascending=False)
df

,ct_id,date,ac_location,mc_location,name,parent_name,mc_name
6,5,2018-11-06 00:00:00,Germany,United Kingdom,12M Invent GmbH,Mr Kai Martin Seefeldt,12M Limited
7,6,2018-06-20 00:00:00,United Kingdom,Luxembourg,Northern Trust Global Services PLC,Northern Trust Global Services PLC,1889 Holdings S.A.
8,7,2014-07-01 00:00:00,Cyprus,Czech Republic,Mostreet Holdings Limited,Mostreet Holdings Limited,2R Holding s.r.o.
9,8,2017-10-13 00:00:00,Denmark,Germany,3 LG Invest ApS,3 LG Invest ApS,3 LG Invest GmbH
10,9,2014-06-20 00:00:00,Netherlands,Cyprus,Coca-Cola HBC Holdings B.V.,Coca-Cola HBC AG,3E (Cyprus) Limited
16,30,2017-10-24 00:00:00,Netherlands,United Kingdom,ABN AMRO Commercial Finance Holding B.V.,ABN AMRO Commercial Finance Holding B.V.,ABN AMRO Commercial Finance Plc
17,31,2018-03-30 00:00:00,Netherlands,Luxembourg,ABN AMRO Bank N.V.,Staat der Nederlanden,ABN AMRO Financing S.Ã r.l.
18,32,2016-10-11 00:00:00,Netherlands,Belgium,ABN AMRO Lease N.V.,Staat der Nederlanden,ABN AMRO Lease BelgiÃ« N.V.
19,5565,2019-12-17 00:00:00,Netherlands,Germany,ABN AMRO Bank N.V.,Staat der Nederlanden,Prospery GmbH
20,33,2019-01-04 00:00:00,Netherlands,Belgium,ABN AMRO Bank N.V.,ABN AMRO Bank N.V.,ABN AMRO Private Banking Belgium SA/NV


In [12]:
# extract transactions for a specific year and put them in a graph
from collections import defaultdict
y_true_dict = defaultdict()
for year in range(2008,2019):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year+1, 1, 1)
    G = data.df2graph(start_date=start_date, end_date=end_date)
    edge_data = data.edges2dict(G)
    print(len(G.edges), len(edge_data))
    y_true_dict[year] = [edge_data[edge] for edge in edge_data]

57 702
83 702
101 702
114 702
128 702
97 702
94 702
93 702
116 702
127 702
117 702


In [7]:
y_true = [edge_data[edge] for edge in edge_data]
y_pred= [predicted_edge_data[edge] for edge in predicted_edge_data]

In [23]:
pd.DataFrame(y_true_dict)

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,9,5,3,7,12,12,12,5,20,57,30
1,1,5,0,2,6,20,7,8,12,12,8
2,0,1,0,1,2,1,2,3,2,1,2
3,0,0,1,5,6,6,5,2,5,1,7
4,0,1,1,0,3,0,0,3,2,3,3
5,1,2,3,13,8,6,2,4,4,6,3
6,0,0,0,0,1,0,0,1,1,1,0
7,0,0,0,0,0,0,0,1,1,0,0
8,0,0,1,0,2,0,2,0,0,0,0
9,0,0,0,0,1,0,1,1,0,0,0


In [17]:
pickle.dump(y_true_dict, open('C:/Users/Kinga/OneDrive/thesis/data/corpgame/real_edge_payoff.pickle','wb'))

In [6]:
predicted_edge_data = pickle.load(open('C:/Users/Kinga/OneDrive/thesis/data/corpgame/predicted_edge_payoff.pickle','rb'))

In [13]:
y_mean =[np.mean(y_true) for i in range(702)]
y_zero = [0]*702

In [15]:
e1 = mean_squared_error(y_true, y_zero, squared=False)
e2 = mean_squared_error(y_true, y_mean, squared=False)
e3 = mean_squared_error(y_true, y_pred, squared=False)
print(e1, e2, e3)

3.0655237558100734 2.96558788231464 3.0655237558100734


In [22]:
[sum(y_true_dict[year]) for year in y_true_dict] #this should be a early amount of CBMs!

[0.2037037037037037,
 0.2934472934472934,
 0.37464387464387466,
 0.4415954415954416,
 0.5042735042735043,
 0.42165242165242167,
 0.35185185185185186,
 0.2962962962962963,
 0.43874643874643876,
 0.5854700854700855,
 0.5868945868945868]